# Demonstration and Testing of the Config() class

The purpose of the Config class is to offer a lightweight and easy-to-use configuration system. Conceptually, configurations are (optionally-nested) dictionaries. This class can represent and output the configuration as a dictionary or as a possibly more-readable YAML file. This notebook walks through some of the core funtionality of the class.

## Imports and Setup

In [1]:
import os
os.chdir('/home/djliden91/git/projects/numerai')
from src.config import config
import yaml
from pathlib import Path

## Example Configurations
We will be working with two configuration files: `dafault_config` and `update_config`. When a `Config` instance is updated with a new configuration file, any new key/value pairs are added to the configuration. If a new configuration contains keys also contained in the existing configuration, the values are updated to match the new configuration file. Old keys not included in the new configuration are left as they are.

Both of these samples start out as `yaml` files, but we can just as easily start with python dictionaries.

In [2]:
default_config = Path("./src/config/default_config.yaml")
update_config = Path("./src/config/experiments/config_debug.yaml")

In [3]:
yaml.load(open(default_config, 'r'), Loader=yaml.SafeLoader)

{'SYSTEM': {'DEBUG': False},
 'DATA': {'REFRESH': False, 'SAVE_PROCESSED_TRAIN': True},
 'EVAL': {'SAVE_PREDS': True, 'SUBMIT_PREDS': False},
 'MODEL': {'NAME': 'Unnamed'}}

In [4]:
yaml.load(open(update_config, 'r'), Loader=yaml.SafeLoader)

{'MODEL': {'NAME': 'config_debug', 'WEIGHT_DECAY': 0.5, 'LAYERS': [400, 200]},
 'EVAL': {'SAVE_PREDS': False},
 'TRAIN': {'N_EPOCHS': 2}}

Some key differences between these configurations to pay attention to:
* The "default" name is `Unnamed` while the "update" name is `config_debug`
* The "default" config does not include a `TRAIN` key.
* The "update" config does not include a `SYSTEM` key.

## Initialize our Configuration
We can pass one of our paths to a yaml config to the `Config` class to get started.

In [5]:
cfg = config.Config(default_config)

### Useful Methods
Before getting into the addition of new configurations and some of the additional features of this system, we'll go over the "core functionality." We can:
1. View our whole config using the `config` attribute

In [6]:
cfg.config

{'SYSTEM': {'DEBUG': False},
 'DATA': {'REFRESH': False, 'SAVE_PROCESSED_TRAIN': True},
 'EVAL': {'SAVE_PREDS': True, 'SUBMIT_PREDS': False},
 'MODEL': {'NAME': 'Unnamed'}}

Or we can view it in YAML formatting using `dump_config`. If we pass a path argument to this method, it will save the output as a `yaml` file at the location indicated.

In [7]:
cfg.dump_config()

DATA:
  REFRESH: false
  SAVE_PROCESSED_TRAIN: true
EVAL:
  SAVE_PREDS: true
  SUBMIT_PREDS: false
MODEL:
  NAME: Unnamed
SYSTEM:
  DEBUG: false



We can use dot notation to reference individual elements of the configuration or to print "nodes" of the configuration. Eventually, it would probably be useful to make sure the actual nested dictionaries are returned when their nodes are called.

In [8]:
cfg.DATA.REFRESH

False

In [9]:
cfg.DATA

Config Object with Keys:
REFRESH: false
SAVE_PROCESSED_TRAIN: true

The corresponding dictionary can be obtained with:

In [10]:
cfg.DATA.config

{'REFRESH': False, 'SAVE_PROCESSED_TRAIN': True}

## Update the Configuration

Once initialized, the configuration can be updated with new dictionaries or yaml files. This can be used to add new configuration options or overwrite existing ones.

In [11]:
cfg.MODEL.NAME

'Unnamed'

In [12]:
cfg.update_config(update_config)
cfg.MODEL.NAME

'config_debug'

In [13]:
cfg

Config Object with Keys:
DATA:
  REFRESH: false
  SAVE_PROCESSED_TRAIN: true
EVAL:
  SAVE_PREDS: false
MODEL:
  LAYERS:
  - 400
  - 200
  NAME: config_debug
  WEIGHT_DECAY: 0.5
SYSTEM:
  DEBUG: false
TRAIN:
  N_EPOCHS: 2

The only way to add to the config is via the `update_config` method. If we try to add a new configuration option, we will get an error:

In [14]:
cfg.COMMENT = "I'm trying to add a comment with dot notation"

TypeError:  Directly adding new attributes to a Configuration  object using dot
notation is not supported. Please use  the update_config() method

Lastly, a small example showing how we might use the configuration.

In [15]:
cfg_dict = {'name':'Dan', 'n_apples':12, 'wallet':{'dollars':8, 'cents':15}}
cfg = config.Config(cfg_dict)

In [18]:
(f'{cfg.name} has {cfg.n_apples} apples. He can sell the apples for $1.00 each. '
 f'After he sells the apples, he will have {cfg.wallet.dollars + cfg.n_apples} '
 f'dollars and {cfg.wallet.cents} cents')

'Dan has 12 apples. He can sell the apples for $1.00 each. After he sells the apples, he will have 20 dollars and 15 cents'